In [ ]:
pip install mlforecast==0.9.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
pip install Seance --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
from Seance.Optimizer import Optimize
from Seance.Forecaster import Forecaster
import time
import pandas as pd

# Weekly Dataset

In [ ]:
weekly_train = pd.read_csv("/content/weekly_m4_train.csv")
weekly_test = pd.read_csv("/content/weekly_m4_test.csv")

weekly_train['Datetime'] = pd.to_datetime(weekly_train['Datetime'])
weekly_test['Datetime'] = pd.to_datetime(weekly_test['Datetime'])

timeout = 60 * 60 *1

In [ ]:
tic = time.perf_counter()
opt = Optimize(weekly_train[['V', 'Datetime', 'ID']],
              target_column='V',
              date_column='Datetime',
              id_column='ID',
              freq='W',
              metric='smape',
              seasonal_period=[52],
              test_size=13,
               model='xgboost',
              timeout=timeout)
best_params, study = opt.fit(seed=1)
toc = time.perf_counter()

[I 2023-10-16 19:19:51,795] A new study created in memory with name: no-name-fa9f892a-ce09-4169-bca8-abf5e4a02e07
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [52] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:321: FutureWarning: Not prepending group keys

In [ ]:
print(toc - tic)

1329.2369046830004


In [ ]:
seance = Forecaster(floor=0)
seance.fit(weekly_train[['V', 'Datetime', 'ID']],
           target_column='V',
           date_column='Datetime',
           id_column='ID',
           freq='W',
           **best_params)

predicted = seance.predict(13)

/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:159: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  basis = self.proce

In [ ]:
predicted['sort_id'] = predicted['ID'].str[1:].astype(int)
predicted = predicted.sort_values(['sort_id', 'Datetime'])
predicted = predicted.merge(weekly_test, on=['ID', 'Datetime'])
import numpy as np
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (0.0001 + (np.abs(A) + np.abs(F))))
def grouped_smape(df):
    return smape(df['V'], df[seance.pred_col])
test = predicted.groupby('ID').apply(grouped_smape)
print(np.mean(test))

7.983553942742777


# Yearly Dataset

In [ ]:
yearly_train = pd.read_csv("/content/yearly_m4_train.csv")
yearly_test = pd.read_csv("/content/yearly_m4_test.csv")

yearly_train['Datetime'] = pd.to_datetime(yearly_train['Datetime'])
yearly_test['Datetime'] = pd.to_datetime(yearly_test['Datetime'])

timeout = 60 * 60 * 2

In [ ]:
tic = time.perf_counter()
opt = Optimize(yearly_train[['V', 'Datetime', 'ID']],
            target_column='V',
            date_column='Datetime',
            id_column='ID',
            freq='H',
            metric='smape',
               scale_types=['minmax', 'standard', 'none'],
            test_size=6,
               model='xgboost',
            timeout=timeout)
best_params, study = opt.fit(seed=1)
toc = time.perf_counter()
print(toc - tic)

[I 2023-10-16 23:25:26,804] A new study created in memory with name: no-name-3a0b75f5-a500-4d14-bb2f-4aa830338ca4
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
[I 2023-10-16 23:25:54,434] Trial 0 finished with value: inf and parameters: {'use_id': False, 'n_basis': 0

ERROR WHILE TUNING: [23:25:54] /workspace/src/data/iterative_dmatrix.cc:202: Check failed: n_features >= 1 (0 vs. 1) : Data must has at least 1 column.
Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3ef8ea) [0x7a299e1588ea]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3f52e7) [0x7a299e15e2e7]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3f8188) [0x7a299e161188]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3a2367) [0x7a299e10b367]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGQuantileDMatrixCreateFromCallback+0x2b0) [0x7a299deceab0]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7a2a2ecd8e2e]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7a2a2ecd5493]
  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa3e9) [0x7a2a2ecfe3e9]
  [bt] (8) /usr/lib/python3

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:159: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardle

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-16 23:27:37,645] Trial 1 finished with value: 24.74545490731167 and parameters: {'use_id': True, 'n_basis': 10, 'differences': None, 'decay': 0.9, 'scale_type': 'standard', 'n_estimators': 251, 'max_depth': 10, 'learning_rate': 0.0038657013301325246, 'subsample': 0.3589978047277139, 'bagging_freq': 0.2170257149064499, 'colsample_bytree': 0.11743026208326737, 'min_data_in_leaf': 68.20471776104921, 'reg_lambda': 4.93202112372755e-07, 'reg_alpha': 1.3315984316584491e-06, 'min_child_weight': 6, 'seasonal_period': [0], 'fourier_order': 6, 'lags': 1

...

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-17 01:24:24,582] Trial 85 finished with value: 13.358862043913986 and parameters: {'use_id': False, 'n_basis': 25, 'differences': 1, 'decay': -1, 'scale_type': 'minmax', 'n_estimators': 376, 'max_depth': 9, 'learning_rate': 0.003641272928184868, 'subsample': 0.2376881054348085, 'bagging_freq': 0.543485968867934, 'colsample_bytree': 0.21042033975546803, 'min_data_in_leaf': 96.66762144876374, 'reg_lambda': 8.61572833240231e-05, 'reg_alpha': 0.5493738018183719, 'min_child_weight': 9, 'seasonal_period': [0], 'fourier_order': 14, 'lags': 1}. Best i

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-17 01:26:12,211] Trial 86 finished with value: 13.529236832008277 and parameters: {'use_id': False, 'n_basis': 25, 'differences': 1, 'decay': 0.25, 'scale_type': 'minmax', 'n_estimators': 372, 'max_depth': 9, 'learning_rate': 0.003636767844781739, 'subsample': 0.236228382798922, 'bagging_freq': 0.5990194720445134, 'colsample_bytree': 0.2055872054560955, 'min_data_in_leaf': 97.37265511794938, 'reg_lambda': 7.818371910701738e-05, 'reg_alpha': 0.559835992314896, 'min_child_weight': 10, 'seasonal_period': [0], 'fourier_order': 14, 'lags': 1}. Best

7245.927899233


In [ ]:
seance = Forecaster(floor=0)
seance.fit(yearly_train[['V', 'Datetime', 'ID']],
           target_column='V',
           date_column='Datetime',
           id_column='ID',
           freq='H',
           **best_params
           )

predicted = seance.predict(6)

/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:159: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  basis = self.proce

In [ ]:
predicted['sort_id'] = predicted['ID'].str[1:].astype(int)
predicted = predicted.sort_values(['sort_id', 'Datetime'])
predicted = predicted.merge(yearly_test, on=['ID', 'Datetime'])
import numpy as np
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (0.0001 + (np.abs(A) + np.abs(F))))
def grouped_smape(df):
    return smape(df['V'], df[seance.pred_col])
test = predicted.groupby('ID').apply(grouped_smape)
print(np.mean(test))

13.61422195241449


In [ ]:
print(best_params)

{'use_id': False, 'n_basis': 25, 'differences': 1, 'decay': -1, 'scale_type': 'minmax', 'n_estimators': 335, 'max_depth': 7, 'learning_rate': 0.004493071935290441, 'subsample': 0.2723337921415635, 'bagging_freq': 0.5245617698596212, 'colsample_bytree': 0.24147129134311074, 'min_data_in_leaf': 81.86877885774175, 'reg_lambda': 0.00022393164270118008, 'reg_alpha': 0.9102118299993066, 'min_child_weight': 9, 'seasonal_period': [0], 'fourier_order': 11, 'lags': [1], 'model': 'xgboost'}


# Monthly Dataset

In [ ]:
monthly_train = pd.read_csv("/content/monthly_m4_train.csv")
monthly_test = pd.read_csv("/content/monthly_m4_test.csv")

monthly_train['Datetime'] = pd.to_datetime(monthly_train['Datetime'])
monthly_test['Datetime'] = pd.to_datetime(monthly_test['Datetime'])
timeout = 60 * 60 * 12

In [ ]:
tic = time.perf_counter()
opt = Optimize(monthly_train[['V', 'Datetime', 'ID']],
            target_column='V',
            date_column='Datetime',
            id_column='ID',
            freq='M',
            metric='smape',
               scale_types=['minmax', 'standard', 'none'],
            seasonal_period=[12],
            test_size=6,
               model='xgboost',
            timeout=timeout)
best_params, study = opt.fit(seed=1)
toc = time.perf_counter()
print(toc - tic)

[I 2023-10-19 20:04:07,751] A new study created in memory with name: no-name-84a70168-a732-49b2-8d4b-99880f2c1e33
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [12] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:321: FutureWarning: Not prepending group keys

27339.50037405


In [ ]:
seance = Forecaster(floor=0)
seance.fit(monthly_train[['V', 'Datetime', 'ID']],
           target_column='V',
           date_column='Datetime',
           id_column='ID',
           freq='M',
           **best_params
           )

predicted = seance.predict(18)

/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:152: UserWarning: The differences argument is deprecated and will be removed in a future version.
Please pass an `mlforecast.target_transforms.Differences` instance to the `target_transforms` argument instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:321: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be includ

In [ ]:
predicted['sort_id'] = predicted['ID'].str[1:].astype(int)
predicted = predicted.sort_values(['sort_id', 'Datetime'])
predicted = predicted.merge(monthly_test, on=['ID', 'Datetime'])
import numpy as np
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (0.0001 + (np.abs(A) + np.abs(F))))
def grouped_smape(df):
    return smape(df['V'], df[seance.pred_col])
test = predicted.groupby('ID').apply(grouped_smape)
print(np.mean(test))

13.621507011146651


In [ ]:
print(np.shape(monthly_test))
print(np.shape(predicted))

(864000, 3)
(864000, 6)


# Quarterly Dataset

In [ ]:
quarterly_train = pd.read_csv("/content/quarterly_m4_train.csv")
quarterly_test = pd.read_csv("/content/quarterly_m4_test.csv")

quarterly_train['Datetime'] = pd.to_datetime(quarterly_train['Datetime'])
quarterly_test['Datetime'] = pd.to_datetime(quarterly_test['Datetime'])
timeout = 60 * 60 * 4

In [ ]:
tic = time.perf_counter()
opt = Optimize(quarterly_train[['V', 'Datetime', 'ID']],
            target_column='V',
            date_column='Datetime',
            id_column='ID',
            freq='Q',
            metric='smape',
            scale_types=['none', 'minmax', 'standard'],
            test_size=8,
               model='xgboost',
            timeout=timeout)
best_params, study = opt.fit(seed=1)
toc = time.perf_counter()
print(toc - tic)

[I 2023-10-17 01:28:16,718] A new study created in memory with name: no-name-f611a116-4acb-4e64-a013-819a345f9a8e
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0] which is of type list.
  warnings.warn(message)
[I 2023-10-17 01:28:20,092] Trial 0 finished with value: inf and parameters: {'use_id': False, 'n_basis': 0, 'differences': None, 'decay': 0.9, 'scale_type': 'none', 'n_estimators': 238, 'max_depth': 6, 'learning_rate': 0.0019088591198098556, 'subsample': 0.2782913401763909, 'bagging_freq': 0.820670111807983, 'colsample_bytree': 0.9714354181474578, 'min_data_in_leaf': 32.02899363776504, 'reg_lambda': 0.003456057228897718, 'reg_alpha': 0.10259193433611967, 'min_child_weight': 10, 'seasonal_period': None, 'fourier_order': 6, 'lags': 1}. Best is trial 0 with value: inf.


ERROR WHILE TUNING: [01:28:20] /workspace/src/data/iterative_dmatrix.cc:202: Check failed: n_features >= 1 (0 vs. 1) : Data must has at least 1 column.
Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3ef8ea) [0x7a299e1588ea]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3f52e7) [0x7a299e15e2e7]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3f8188) [0x7a299e161188]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x3a2367) [0x7a299e10b367]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGQuantileDMatrixCreateFromCallback+0x2b0) [0x7a299deceab0]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7a2a2ecd8e2e]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7a2a2ecd5493]
  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa3e9) [0x7a2a2ecfe3e9]
  [bt] (8) /usr/lib/python3

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/Seance/Forecaster.py:159: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardle

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-17 01:30:37,617] Trial 1 finished with value: 24.846021992476846 and parameters: {'use_id': True, 'n_basis': 10, 'differences': None, 'decay': 0.9, 'scale_type': 'minmax', 'n_estimators': 251, 'max_depth': 10, 'learning_rate': 0.0038657013301325246, 'subsample': 0.3589978047277139, 'bagging_freq': 0.2170257149064499, 'colsample_bytree': 0.11743026208326737, 'min_data_in_leaf': 68.20471776104921, 'reg_lambda': 4.93202112372755e-07, 'reg_alpha': 1.3315984316584491e-06, 'min_child_weight': 6, 'seasonal_period': [0], 'fourier_order': 6, 'lags': 1}

...

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-17 04:20:30,062] Trial 98 finished with value: 11.824844751433146 and parameters: {'use_id': False, 'n_basis': 2, 'differences': 1, 'decay': 0.5, 'scale_type': 'minmax', 'n_estimators': 230, 'max_depth': 7, 'learning_rate': 0.09670595916394292, 'subsample': 0.33086324162604025, 'bagging_freq': 0.48370139718025956, 'colsample_bytree': 0.9636770186004241, 'min_data_in_leaf': 81.89090223488094, 'reg_lambda': 0.4284611961968792, 'reg_alpha': 0.05181456553499802, 'min_child_weight': 2, 'seasonal_period': [0], 'fourier_order': 16, 'lags': 1}. Best i

Predicted not the same size as test set


/usr/local/lib/python3.10/dist-packages/Seance/Optimizer.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.test_df[self.date_column] = self.test_df[self.date_column].dt.tz_localize(None)
[I 2023-10-17 04:22:14,547] Trial 99 finished with value: 11.751454529944365 and parameters: {'use_id': False, 'n_basis': 3, 'differences': 1, 'decay': 0.25, 'scale_type': 'minmax', 'n_estimators': 295, 'max_depth': 7, 'learning_rate': 0.06425663658582634, 'subsample': 0.4759785572477374, 'bagging_freq': 0.5652662464898497, 'colsample_bytree': 0.8865875006335162, 'min_data_in_leaf': 88.91443775946374, 'reg_lambda': 0.04243295124582627, 'reg_alpha': 0.04283693699630789, 'min_child_weight': 2, 'seasonal_period': [0], 'fourier_order': 6, 'lags': 1}. Best is

10439.217586982002


In [ ]:
seance = Forecaster(floor=0)
seance.fit(quarterly_train[['V', 'Datetime', 'ID']],
           target_column='V',
           date_column='Datetime',
           id_column='ID',
           freq='Q',
           **best_params
           )

predicted = seance.predict(8)

/usr/local/lib/python3.10/dist-packages/Seance/Builder/PreProcess.py:81: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Seance ID').apply(self.trans)
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:152: UserWarning: The differences argument is deprecated and will be removed in a future version.
Please pass an `mlforecast.target_transforms.Differences` instance to the `target_transforms` argument instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:476: UserWarning: Found null values in 0_fourier_1, 0_fourier_2, 0_fourier_3, 0_fourier_4, 0_fourier_5, 0_fourier_6, 0_fourier_7, 0_fouri

In [ ]:
print(predicted['Datetime'].sort_values().unique())

['1894-03-31T00:00:00.000000000' '1894-06-30T00:00:00.000000000'
 '1894-09-30T00:00:00.000000000' '1894-12-31T00:00:00.000000000'
 '1895-03-31T00:00:00.000000000' '1895-06-30T00:00:00.000000000'
 '1895-09-30T00:00:00.000000000' '1895-12-31T00:00:00.000000000']


In [ ]:
print(quarterly_test['Datetime'].sort_values().unique())

['1894-03-31T00:00:00.000000000' '1894-06-30T00:00:00.000000000'
 '1894-09-30T00:00:00.000000000' '1894-12-31T00:00:00.000000000'
 '1895-03-31T00:00:00.000000000' '1895-06-30T00:00:00.000000000'
 '1895-09-30T00:00:00.000000000' '1895-12-31T00:00:00.000000000']


In [ ]:
predicted['sort_id'] = predicted['ID'].str[1:].astype(int)
predicted = predicted.sort_values(['sort_id', 'Datetime'])
predicted = predicted.merge(quarterly_test, on=['ID', 'Datetime'])
import numpy as np
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (0.0001 + (np.abs(A) + np.abs(F))))
def grouped_smape(df):
    return smape(df['V'], df[seance.pred_col])
test = predicted.groupby('ID').apply(grouped_smape)
print(np.mean(test))

11.039004277543757


# Hourly Dataset

In [ ]:
hourly_train = pd.read_csv("/content/hourly_m4_train.csv")
hourly_test = pd.read_csv("/content/hourly_m4_test.csv")

hourly_train['Datetime'] = pd.to_datetime(hourly_train['Datetime'])
hourly_test['Datetime'] = pd.to_datetime(hourly_test['Datetime'])
timeout = 60 * 60 * 3

In [ ]:
tic = time.perf_counter()
opt = Optimize(hourly_train[['V', 'Datetime', 'ID']],
            target_column='V',
            date_column='Datetime',
            id_column='ID',
            freq='H',
            metric='smape',
            seasonal_period=[[24,168], 24],
            test_size=48,
               model='xgboost',
            timeout=timeout)
best_params, study = opt.fit(seed=1)
toc = time.perf_counter()
print(toc - tic)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:355: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat] = features[feat][keep_rows]
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:355: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat] = features[feat][keep_rows]
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:355: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

6238.842879722


In [ ]:
seance = Forecaster(floor=0)
seance.fit(hourly_train[['V', 'Datetime', 'ID']],
           target_column='V',
           date_column='Datetime',
           id_column='ID',
           freq='H',
           **best_params
           )

predicted = seance.predict(48)

In [ ]:
predicted['sort_id'] = predicted['ID'].str[1:].astype(int)
predicted = predicted.sort_values(['sort_id', 'Datetime'])
predicted = predicted.merge(hourly_test, on=['ID', 'Datetime'])
import numpy as np
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (0.0001 + (np.abs(A) + np.abs(F))))
def grouped_smape(df):
    return smape(df['V'], df[seance.pred_col])
test = predicted.groupby('ID').apply(grouped_smape)
print(np.mean(test))